In [ ]:
pip install folium

pip install haversine



In [66]:

import xml.etree.ElementTree as ET
import pprint
import random as rd
import folium as flMap
from haversine import haversine
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix



In [75]:

anonimity_threshold = 3


class UserObject:
    def __init__(self,id,lat_lon_obj,wsscr=None):
        self.id = id
        self.lat_lon_obj = lat_lon_obj
        self.wsscr=wsscr

class LatLon:
    def __init__(self,lat,lon):
        self.lat = float(lat)
        self.lon = float(lon)


class WSSCR:
    cloaked_search_space_radius= 1  # kilo meters
    def __init__(self,WSSCR_list,WSSCR_id):
        self.id = WSSCR_id
        self.wsscr_users = WSSCR_list


# def DecisionTree():

# parsing xml file for lat lng

tree = ET.parse('route_2.gpx')
root = tree.getroot()

#print(root.tag)

lat_lon_list=[]
for i in root.iter("wpt"):
    lat_lon_list.append(LatLon(i.attrib['lat'],i.attrib['lon']))

#pprint.PrettyPrinter().pprint(lat_lon_list)



In [77]:
    

def userIdGenerator():
    id=rd.randint(1,1000)
    return "user" + str(id)

def addWSSCR(user,userList):
    t_diff=0
    radius = WSSCR.cloaked_search_space_radius
    min = float('inf')
    for i in userList:
        diff = round(haversine((user.lat_lon_obj.lat,user.lat_lon_obj.lon),(i.lat_lon_obj.lat,i.lat_lon_obj.lon)),3)
        t_diff=diff
        #print(diff)
        if(diff<radius):
            if(diff<min):
                if(i.wsscr):
                    user.wsscr = i.wsscr
    if(user.wsscr==None):
        user.wsscr = "wsscr:" + str(user.id) +str(rd.randint(1,10))
    
    #print(t_diff,user.wsscr)


userList = []
temp_list = lat_lon_list.copy()
for i in lat_lon_list:
    userLatLon =  rd.choice(temp_list)
    user = UserObject(userIdGenerator(),userLatLon)
    userList.append(user)    
    temp_list.remove(i)

for i in userList:
    addWSSCR(i,userList)
    



#pprint.PrettyPrinter().pprint(userList)
# for i in userList:
#     print(i.name,i.lat_lon_obj)
    
# map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=15, tiles="Stamen Terrain")

# for i in userList:
#     flMap.Marker([i.lat_lon_obj.lat,i.lat_lon_obj.lon],popup=f"{i.lat_lon_obj.lat},{i.lat_lon_obj.lon}",tooltip=f"<i>{i.id}</i>").add_to(map)
    
# map


# #pprint.PrettyPrinter().pprint(userList)
# # for i in userList:
# #     print(i.name,i.lat_lon_obj)
    
# map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=14, tiles="Stamen Terrain")

# for i in userList:
#     flMap.Marker([i.lat_lon_obj.lat,i.lat_lon_obj.lon],popup=f"{i.wsscr}, {i.lat_lon_obj.lat},{i.lat_lon_obj.lon}",tooltip=f"<i>{i.id}</i>").add_to(map)
    
# map

# wsscr_list=[]

# for i in userList:
#     if(i.wsscr in wsscr_list):
#         continue

#     wsscr_list.append(i.wsscr)
#     flMap.Circle(
#         location=[i.lat_lon_obj.lat,i.lat_lon_obj.lon],
#         radius=1000,
#         popup=i.wsscr,
#         color="#3186cc",
#         fill=True,
#         fill_color="#3186cc",
#     ).add_to(map)

# map


# # Input user  Object
# #lat="30.88321" lon="75.79903"
# UserObjectForTest = UserObject(userIdGenerator(),LatLon(30.88321,75.79903))


#pprint.PrettyPrinter().pprint(userList)
# for i in userList:
#     print(i.name,i.lat_lon_obj)
    


In [78]:
map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=14, tiles="Stamen Terrain")

for i in userList:
    flMap.Marker([i.lat_lon_obj.lat,i.lat_lon_obj.lon],popup=f"{i.wsscr}, {i.lat_lon_obj.lat},{i.lat_lon_obj.lon}",tooltip=f"<i>{i.id}</i>").add_to(map)

wsscr_list=[]

for i in userList:
    if(i.wsscr in wsscr_list):
        continue
    wsscr_list.append(i.wsscr)
    flMap.Circle(
        location=[i.lat_lon_obj.lat,i.lat_lon_obj.lon],
        radius=1000*WSSCR.cloaked_search_space_radius,
        popup=i.wsscr,
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
    ).add_to(map)

# Input user  Object
#lat="30.87374" lon="75.81251"
#u1 = UserObject(userIdGenerator(),LatLon(30.88321,75.79903))

u1 = UserObject("testing User",LatLon(30.87374,75.81251))

#flMap.Marker([u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],popup=f"{u1.wsscr}, {u1.lat_lon_obj.lat},{u1.lat_lon_obj.lon}",tooltip=f"<i>{u1.id}</i>").add_to(map)

#showing map
map



In [79]:

#implementin knn algorithm
  
# Loading data
temp_list = [[user.id,user.lat_lon_obj.lat,user.lat_lon_obj.lon,user.wsscr] for user in userList]
dataset = pd.DataFrame(temp_list,columns=["User ID","Lat","Lon","WSSCR ID"])

#print(dataset)

# # Create feature and target arrays
x = dataset.iloc[:,1:3].values
y = dataset.iloc[:,-1].values

#print(x)
#print(y)
  
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=10)

In [80]:
# print(knn.predict(X_test))

y_pred = knn.predict(X_test)
#print(y_pred)


In [81]:

print(classification_report(y_test,y_pred))
#print(confusion_matrix(y_test,y_pred))


                precision    recall  f1-score   support

wsscr:user4112       0.00      0.00      0.00         0
wsscr:user9888       1.00      0.75      0.86         4
wsscr:user9997       1.00      0.50      0.67         2

      accuracy                           0.67         6
     macro avg       0.67      0.42      0.51         6
  weighted avg       1.00      0.67      0.79         6



c:\Users\rahul\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\rahul\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\rahul\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [82]:
data =[[u1.lat_lon_obj.lat,u1.lat_lon_obj.lon]]

predicted_wsscr = knn.predict(data)
u1.wsscr= predicted_wsscr
print(u1.wsscr)

print(wsscr_list)
flMap.Marker([u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],
                color='green',
                icon=flMap.Icon(color='green', icon='info-sign'),
                popup=f"{u1.wsscr}, {u1.lat_lon_obj.lat},{u1.lat_lon_obj.lon}",
                tooltip=f"<i>{u1.id}</i>").add_to(map)


#avg_lat_lon_obj = [user for i in userList]
# flMap.Circle(
#         location=[u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],
#         radius=1000,
#         popup=u1.wsscr,
#         color="#3186cc",
#         fill=True,
#         fill_color="#FF0000",
#     ).add_to(map)

map

['wsscr:user9888']
['wsscr:user972', 'wsscr:user9888', 'wsscr:user494', 'wsscr:user4375', 'wsscr:user6248', 'wsscr:user44210', 'wsscr:user9997', 'wsscr:user89710', 'wsscr:user7198', 'wsscr:user4112']


In [64]:
wsscr_dict = {}

#pprint.PrettyPrinter().pprint(wsscr_list)
# print(len(userList))
# print(len(wsscr_list))

# for t in wsscr_list:
#     count = 0
#     for j in userList:
#         if(j.wsscr == t):
#             count += 1
#     print(t,"  <> ",count)


for i in wsscr_list:
    count = 0
    wsscr_dict[i]=[]
    for j in userList:
        if(j.wsscr == i):
            #count += 1 
            wsscr_dict[i].append(j)

pprint.PrettyPrinter().pprint(wsscr_dict)

# testing

# for t in wsscr_dict:
#     print(len(wsscr_dict[t]))




{'wsscr:user2807': [<__main__.UserObject object at 0x000001E1A4219030>,
 'wsscr:user3995': [<__main__.UserObject object at 0x000001E1A42198A0>],
 'wsscr:user4034': [<__main__.UserObject object at 0x000001E1A421A2C0>,
 'wsscr:user4276': [<__main__.UserObject object at 0x000001E1A4219840>],
 'wsscr:user5681': [<__main__.UserObject object at 0x000001E1A4219390>,
 'wsscr:user6110': [<__main__.UserObject object at 0x000001E1A4219900>,
 'wsscr:user7159': [<__main__.UserObject object at 0x000001E1A4219BA0>,
 'wsscr:user8375': [<__main__.UserObject object at 0x000001E1A42190F0>],
 'wsscr:user8869': [<__main__.UserObject object at 0x000001E1A421AF20>,
 'wsscr:user9194': [<__main__.UserObject object at 0x000001E1A421AC80>]}
